## 加法秘密共享


### 元素拆分

In [1]:
# 构造简单n方的加法秘密共享

import random

def secret_share(secret, n):
    """
    将秘密分成n个部分，每个部分相加可以还原秘密
    :param secret: 要分享的秘密 (整数)
    :param n: 参与方的数量
    :return: 包含n个部分的列表
    """
    shares = [random.randint(0, secret) for _ in range(n - 1)]
    shares.append(secret - sum(shares))
    return shares

def reconstruct_secret(shares):
    """
    通过分享的部分还原秘密
    :param shares: 包含n个部分的列表
    :return: 还原的秘密 (整数)
    """
    return sum(shares)

# 示例用法
secret = 12345
n = 5
shares = secret_share(secret, n)
print("Shares:", shares)
reconstructed_secret = reconstruct_secret(shares)
print("Reconstructed Secret:", reconstructed_secret)

Shares: [6335, 6972, 2944, 6162, -10068]
Reconstructed Secret: 12345


### 向量拆分

In [2]:
# 构造简单n方的加法秘密共享——向量版本

import random

def secret_share_vector(secret_vector, n, rand_range):
    """
    将秘密向量分成n个部分，每个部分相加可以还原秘密向量
    :param secret_vector: 要分享的秘密向量 (列表)
    :param n: 参与方的数量
    :param rand_range: 随机数生成范围 (元组)
    :return: 包含n个部分的列表，每个部分也是一个向量
    """
    shares = []
    for secret in secret_vector:
        share = [random.randint(rand_range[0], rand_range[1]) for _ in range(n - 1)]
        share.append(secret - sum(share))
        shares.append(share)
    return list(map(list, zip(*shares)))

def reconstruct_secret_vector(shares):
    """
    通过分享的部分还原秘密向量
    :param shares: 包含n个部分的列表，每个部分也是一个向量
    :return: 还原的秘密向量 (列表)
    """
    return [sum(share) for share in zip(*shares)]

# 示例用法
secret_vector = [12345, 67890, 111213]
n = 5
rand_range = (0, 10000)
shares = secret_share_vector(secret_vector, n, rand_range)
print("Shares:", shares)
reconstructed_secret_vector = reconstruct_secret_vector(shares)
print("Reconstructed Secret Vector:", reconstructed_secret_vector)

Shares: [[9555, 7292, 5545], [788, 5576, 2368], [6497, 8847, 4357], [4413, 6572, 4150], [-8908, 39603, 94793]]
Reconstructed Secret Vector: [12345, 67890, 111213]


### 矩阵拆分


In [2]:
# 构造简单n方的加法秘密共享——矩阵版本

import random

def secret_share_matrix(secret_matrix, n, rand_range):
    """
    将秘密矩阵分成n个部分，每个部分相加可以还原秘密矩阵
    :param secret_matrix: 要分享的秘密矩阵 (二维列表)
    :param n: 参与方的数量
    :param rand_range: 随机数生成范围 (元组)
    :return: 包含n个部分的列表，每个部分也是一个矩阵
    """
    shares = [[[] for _ in range(len(secret_matrix[0]))] for _ in range(n)]
    for row in secret_matrix:
        for secret in row:
            share = [random.uniform(rand_range[0], rand_range[1]) for _ in range(n - 1)]
            share.append(secret - sum(share))
            for i in range(n):
                shares[i][secret_matrix.index(row)].append(share[i])
    return shares

def reconstruct_secret_matrix(shares):
    """
    通过分享的部分还原秘密矩阵
    :param shares: 包含n个部分的列表，每个部分也是一个矩阵
    :return: 还原的秘密矩阵 (二维列表)
    """
    rows = len(shares[0])
    cols = len(shares[0][0])
    secret_matrix = [[0] * cols for _ in range(rows)]
    for i in range(rows):
        for j in range(cols):
            secret_matrix[i][j] = sum(share[i][j] for share in shares)
    return secret_matrix

def print_matrix(matrix, title):
    """
    美观地打印矩阵
    :param matrix: 要打印的矩阵 (二维列表)
    :param title: 矩阵的标题 (字符串)
    """
    print(title)
    for row in matrix:
        print(" ".join(f"{val:.2f}" for val in row))
    print()

# 示例用法
secret_matrix = [
    [1.1, 2.2, 3.3],
    [4.4, 5.5, 6.6],
    [7.7, 8.8, 9.9]
]
n = 2
rand_range = (0.0, 10.0)
shares = secret_share_matrix(secret_matrix, n, rand_range)

# 打印Shares
for i, share in enumerate(shares):
    print_matrix(share, f"Share {i + 1}:")

reconstructed_secret_matrix = reconstruct_secret_matrix(shares)
print_matrix(reconstructed_secret_matrix, "Reconstructed Secret Matrix:")

Share 1:
2.65 2.38 8.63
8.04 6.83 2.09
0.03 4.34 2.46

Share 2:
-1.55 -0.18 -5.33
-3.64 -1.33 4.51
7.67 4.46 7.44

Reconstructed Secret Matrix:
1.10 2.20 3.30
4.40 5.50 6.60
7.70 8.80 9.90



### Share上的加法运算

In [41]:
# Share上的加法运算

class Share:
    def __init__(self, value):
        self.value = value

    @classmethod
    def from_secret_shares(cls, secret_shares):
        # 假设 secret_shares 是一个秘密共享生成的对象列表
        # 这里可以根据具体的 secret_shares 对象的结构进行处理
        return [cls(share) for share in secret_shares]
    
    def __sub__(self, other):
        return Share(self.sub(self.value, other.value))

    def sub(self, a, b):
        if isinstance(a, (int, float)) and isinstance(b, (int, float)):
            return a - b
        elif isinstance(a, list) and isinstance(b, list):
            if all(isinstance(i, list) for i in a) and all(isinstance(i, list) for i in b):
                return self.sub_matrices(a, b)
            else:
                return self.sub_vectors(a, b)
        else:
            raise TypeError("Unsupported types for subtraction")

    def sub_vectors(self, a, b):
        if len(a) != len(b):
            raise ValueError("Vectors must be of the same size")
        return [x - y for x, y in zip(a, b)]

    def sub_matrices(self, a, b):
        if len(a) != len(b) or len(a[0]) != len(b[0]):
            raise ValueError("Matrices must be of the same size")
        return [[x - y for x, y in zip(row_a, row_b)] for row_a, row_b in zip(a, b)]
    
    def __add__(self, other):
        return Share(self.add(self.value, other.value))

    def add(self, a, b):
        if isinstance(a, (int, float)) and isinstance(b, (int, float)):
            return a + b
        elif isinstance(a, (list, Share)) and isinstance(b, (list, Share)):
            if all(isinstance(i, list) for i in a) and all(isinstance(i, list) for i in b):
                return self.add_matrices(a, b)
            else:
                return self.add_vectors(a, b)
        else:
            raise TypeError("Unsupported types for addition")

    def add_vectors(self, a, b):
        if len(a) != len(b):
            raise ValueError("Vectors must be of the same size")
        return [x + y for x, y in zip(a, b)]

    def add_matrices(self, a, b):
        if len(a) != len(b) or len(a[0]) != len(b[0]):
            raise ValueError("Matrices must be of the same size")
        return [[x + y for x, y in zip(row_a, row_b)] for row_a, row_b in zip(a, b)]

    @staticmethod
    def secret_share(secret, n):
        shares = [random.uniform(0, secret) for _ in range(n - 1)]
        shares.append(secret - sum(shares))
        return shares
        # return Share.from_secret_shares(shares)

    @staticmethod
    def reconstruct_secret(shares):
        return sum(shares)
        # return Share.from_secret_shares([sum(shares)])

    @staticmethod
    def secret_share_vector(secret_vector, n, rand_range):
        shares = []
        for secret in secret_vector:
            share = [random.uniform(rand_range[0], rand_range[1]) for _ in range(n - 1)]
            share.append(secret - sum(share))
            shares.append(share)
        return list(map(list, zip(*shares)))
        # return Share.from_secret_shares(list(map(list, zip(*shares))))

    @staticmethod
    def reconstruct_secret_vector(shares):
        return [sum(share) for share in zip(*shares)]
        # return Share.from_secret_shares([sum(share) for share in zip(*shares)])

    @staticmethod
    def secret_share_matrix(secret_matrix, n, rand_range):
        shares = [[[] for _ in range(len(secret_matrix))] for _ in range(n)]
        for row_idx, row in enumerate(secret_matrix):
            for col_idx, secret in enumerate(row):
                share = [random.uniform(rand_range[0], rand_range[1]) for _ in range(n - 1)]
                share.append(secret - sum(share))
                # print(f"shares的维度: {len(shares)} x {len(shares[0])} x {len(shares[0][row_idx])}")
                
                for i in range(n):
                    # print(f"i: {i}, row_idx: {row_idx}, col_idx: {col_idx}, share[i]: {share[i]}")
                    shares[i][row_idx].append(share[i])
        return shares
        # return Share.from_secret_shares(shares)

    @staticmethod
    def reconstruct_secret_matrix(shares):
        rows = len(shares[0])
        cols = len(shares[0][0])
        secret_matrix = [[0] * cols for _ in range(rows)]
        for i in range(rows):
            for j in range(cols):
                secret_matrix[i][j] = sum(share[i][j] for share in shares)
        return secret_matrix
        # return Share.from_secret_shares(secret_matrix)

# 示例用法
# 单个元素
share1 = Share(5)
share2 = Share(10)
result = share1 + share2
print(result.value)  # 输出: 15

# 向量
share3 = Share([1, 2, 3])
share4 = Share([4, 5, 6])
result = share3 + share4
print(result.value)  # 输出: [5, 7, 9]

# 矩阵
share5 = Share([[1, 2], [3, 4]])
share6 = Share([[5, 6], [7, 8]])
result = share5 + share6
print(result.value)  # 输出: [[6, 8], [10, 12]]

15
[5, 7, 9]
[[6, 8], [10, 12]]


In [ ]:
secret_matrix = [
    [1.1, 2.2, 3.3],
    [4.4, 5.5, 6.6],
    [7.7, 8.8, 9.9]
]
n = 2
rand_range = (0.0, 10.0)
shares = Share.secret_share_matrix(secret_matrix, n, rand_range)

print("Shares:")
for i, share in enumerate(shares):
    print_matrix(share, f"Share {i + 1}:")
share_objects = Share.from_secret_shares(shares)

share1 = share_objects[0]
share2 = share_objects[1]
result = share1 + share2
print_matrix(result.value, "Result of Share Addition:")
# print(result)

### Beaver 三元组

In [1]:
import numpy as np

def generate_beaver_triplet(dim1, dim2, dim3):
    """
    随机生成矩阵P、Q、O，其中P*Q=O
    :param dim1: 矩阵P的行数
    :param dim2: 矩阵P的列数和矩阵Q的行数
    :param dim3: 矩阵Q的列数
    :return: 矩阵P、Q、O
    """
    P = np.random.randint(0, 10, (dim1, dim2))
    Q = np.random.randint(0, 10, (dim2, dim3))
    O = np.dot(P, Q)
    return P, Q, O

# 示例用法
dim1, dim2, dim3 = 3, 4, 2
P, Q, O = generate_beaver_triplet(dim1, dim2, dim3)
print("Matrix P:")
print(P)
print("Matrix Q:")
print(Q)
print("Matrix O (P*Q):")
print(O)



Matrix P:
[[0 7 0 8]
 [2 1 2 1]
 [6 0 8 1]]
Matrix Q:
[[8 7]
 [1 2]
 [0 2]
 [6 7]]
Matrix O (P*Q):
[[55 70]
 [23 27]
 [54 65]]


In [27]:
# 将矩阵 P、Q、O 进行秘密拆分，总共拆分成 3 份
n = 3
rand_range = (0.0, 10.0)

# 拆分矩阵 P
shares_P = Share.secret_share_matrix(P.tolist(), n, rand_range)
print("Shares of P:")
for i, share in enumerate(shares_P):
    print_matrix(share, f"Share {i + 1} of P:")

# 拆分矩阵 Q
shares_Q = Share.secret_share_matrix(Q.tolist(), n, rand_range)
print("Shares of Q:")
for i, share in enumerate(shares_Q):
    print_matrix(share, f"Share {i + 1} of Q:")

# 拆分矩阵 O
shares_O = Share.secret_share_matrix(O.tolist(), n, rand_range)
print("Shares of O:")
for i, share in enumerate(shares_O):
    print_matrix(share, f"Share {i + 1} of O:")

Shares of P:
Share 1 of P:
9.17 0.36 8.13 2.48
9.14 2.77 1.21 7.95
6.43 2.84 4.06 9.07

Share 2 of P:
0.70 4.87 6.25 2.44
5.99 3.77 0.15 5.42
6.57 2.74 5.79 2.74

Share 3 of P:
-9.87 1.77 -14.38 3.08
-13.13 -5.54 0.64 -12.37
-7.00 -5.58 -1.84 -10.81

Shares of Q:
Share 1 of Q:
5.96 7.50
5.82 4.11
5.64 4.38
4.96 4.27

Share 2 of Q:
3.48 3.17
5.67 3.93
3.37 9.84
0.48 8.92

Share 3 of Q:
-1.44 -3.67
-10.49 -6.03
-9.01 -12.22
0.57 -6.19

Shares of O:
Share 1 of O:
7.88 0.92
2.09 9.93
2.70 2.56

Share 2 of O:
5.05 1.38
1.64 1.69
1.58 1.07

Share 3 of O:
42.07 67.70
19.28 15.38
49.73 61.37



In [29]:
A = np.random.randint(0, 10, (dim1, dim2))
B = np.random.randint(0, 10, (dim2, dim3))

print("Matrix A:")
print(A)
print("Matrix B:")
print(B)

# 拆分矩阵 A
shares_A = Share.secret_share_matrix(A.tolist(), n, rand_range)
print("Shares of A:")
for i, share in enumerate(shares_A):
    print_matrix(share, f"Share {i + 1} of A:")

# 拆分矩阵 B
shares_B = Share.secret_share_matrix(B.tolist(), n, rand_range)
print("Shares of B:")
for i, share in enumerate(shares_B):
    print_matrix(share, f"Share {i + 1} of B:")

Matrix A:
[[8 0 1 5]
 [9 4 3 7]
 [0 0 9 6]]
Matrix B:
[[7 3]
 [4 1]
 [2 8]
 [5 4]]
Shares of A:
Share 1 of A:
9.50 3.99 3.13 4.01
1.56 8.78 3.01 6.61
8.41 3.27 4.34 7.51

Share 2 of A:
1.45 6.40 9.34 8.91
4.11 5.51 1.53 8.08
4.95 1.96 1.78 1.01

Share 3 of A:
-2.96 -10.39 -11.48 -7.92
3.33 -10.29 -1.54 -7.69
-13.36 -5.23 2.88 -2.53

Shares of B:
Share 1 of B:
1.41 5.85
1.14 3.26
3.93 1.56
9.15 6.83

Share 2 of B:
7.95 6.52
2.60 9.85
9.46 5.00
8.21 5.43

Share 3 of B:
-2.36 -9.37
0.25 -12.11
-11.40 1.44
-12.35 -8.25



In [33]:

# 示例用法
# 将A和B与对应的P和Q进行减法得到E和F的share
shares_E = [Share(a) - Share(p) for a, p in zip(shares_A, shares_P)]
shares_F = [Share(b) - Share(q) for b, q in zip(shares_B, shares_Q)]

print("Shares of E:")
for i, share in enumerate(shares_E):
    print_matrix(share.value, f"Share {i + 1} of E:")

print("Shares of F:")
for i, share in enumerate(shares_F):
    print_matrix(share.value, f"Share {i + 1} of F:")

Shares of E:
Share 1 of E:
0.33 3.63 -4.99 1.53
-7.58 6.01 1.80 -1.33
1.98 0.43 0.29 -1.56

Share 2 of E:
0.75 1.53 3.09 6.47
-1.89 1.74 1.38 2.66
-1.62 -0.78 -4.01 -1.73

Share 3 of E:
6.92 -12.16 2.90 -11.00
16.46 -4.75 -2.18 4.67
-6.36 0.35 4.72 8.29

Shares of F:
Share 1 of F:
-4.55 -1.65
-4.67 -0.85
-1.71 -2.82
4.19 2.55

Share 2 of F:
4.47 3.36
-3.07 5.92
6.10 -4.84
7.73 -3.49

Share 3 of F:
-0.92 -5.71
10.74 -6.08
-2.39 13.66
-12.92 -2.06



In [34]:
# 从share中恢复E和F
E = Share.reconstruct_secret_matrix([share.value for share in shares_E])
F = Share.reconstruct_secret_matrix([share.value for share in shares_F])

print("Matrix E:")
print_matrix(E, "Matrix E:")

print("Matrix F:")
print_matrix(F, "Matrix F:")

# 将E和F进行矩阵相乘
EF = np.dot(E, F)

print("Matrix E * F:")
print_matrix(EF.tolist(), "Matrix E * F:")

Matrix E:
Matrix E:
8.00 -7.00 1.00 -3.00
7.00 3.00 1.00 6.00
-6.00 -0.00 1.00 5.00

Matrix F:
Matrix F:
-1.00 -4.00
3.00 -1.00
2.00 6.00
-1.00 -3.00

Matrix E * F:
Matrix E * F:
-24.00 -10.00
-2.00 -43.00
3.00 15.00



In [39]:
# 计算S的share
shares_S = [Share(np.dot(e.value, q)) for e, q in zip(shares_E, shares_Q)]
print("Shares of S:")
for i, share in enumerate(shares_S):
    print_matrix(share.value, f"Share {i + 1} of S:")

# 计算T的share
shares_T = [Share(np.dot(p, f.value)) for p, f in zip(shares_P, shares_F)]
print("Shares of T:")
for i, share in enumerate(shares_T):
    print_matrix(share.value, f"Share {i + 1} of T:")

Shares of S:
Share 1 of S:
2.45 2.02
-6.66 -29.99
8.19 11.21

Share 2 of S:
24.84 96.57
9.20 38.14
-24.40 -63.06

Share 3 of S:
85.23 80.67
48.35 -34.11
-32.41 -87.81

Shares of T:
Share 1 of T:
-46.89 -32.02
-23.29 -0.55
-11.42 -1.27

Share 2 of T:
45.17 -7.61
58.01 22.81
77.42 0.71

Share 3 of T:
22.70 -157.17
110.81 142.88
90.58 70.98



In [42]:
# 计算G的share
shares_G = [Share(np.dot(e.value, f.value)) for e, f in zip(shares_E, shares_F)]
print("Shares of G:")
for i, share in enumerate(shares_G):
    print_matrix(share.value, f"Share {i + 1} of G:")

# 计算C的share
shares_C = [g + s + t + Share(o) for g, s, t, o in zip(shares_G, shares_S, shares_T, shares_O)]
print("Shares of C:")
for i, share in enumerate(shares_C):
    print_matrix(share.value, f"Share {i + 1} of C:")
    
# 输出shares_G, shares_S, shares_T, shares_O的维度
# print("Dimensions of shares_G:", len(shares_G), "x", len(shares_G[0].value), "x", len(shares_G[0].value[0]))
# print("Dimensions of shares_S:", len(shares_S), "x", len(shares_S[0].value), "x", len(shares_S[0].value[0]))
# print("Dimensions of shares_T:", len(shares_T), "x", len(shares_T[0].value), "x", len(shares_T[0].value[0]))
# print("Dimensions of shares_O:", len(shares_O), "x", len(shares_O[0]), "x", len(shares_O[0][0]))


Shares of G:
Share 1 of G:
-3.51 14.36
-2.29 -1.06
-18.04 -8.43

Share 2 of G:
67.53 -25.96
15.20 -12.02
-42.63 15.37

Share 3 of G:
-1.85 96.70
-121.39 -104.47
-108.66 81.56



TypeError: Unsupported types for addition